In [2]:
#Library installations
import numpy as np

from qiskit import QuantumCircuit, transpile
from qiskit.visualization.counts_visualization import plot_histogram
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from qbraid.runtime.qiskit import QiskitRuntimeProvider
from qbraid.runtime.qiskit.job import QiskitJob

from qiskit_ibm_runtime import QiskitRuntimeService

from dotenv import load_dotenv

import os

In [14]:
API_KEY = os.getenv('IBM_APIKEY')
provider = QiskitRuntimeService(channel='ibm_quantum', token=API_KEY)
qbraid_provider = QiskitRuntimeProvider(runtime_service=provider)
qbraid_provider.get_devices()

[<qbraid.runtime.qiskit.device.QiskitBackend('ibm_brisbane')>,
 <qbraid.runtime.qiskit.device.QiskitBackend('ibm_kyiv')>,
 <qbraid.runtime.qiskit.device.QiskitBackend('ibm_kyoto')>,
 <qbraid.runtime.qiskit.device.QiskitBackend('ibm_sherbrooke')>]

In [15]:
#Building a 100 qubit quantum circuit with all qubits in equal superposition using hadamard gate
circ = QuantumCircuit(100)
for i in range(100):
    circ.h(i)
circ.measure_all()

In [3]:
def launch_job(device_name, shot_count, circ):
    API_KEY = os.getenv('IBM_APIKEY')
    provider = QiskitRuntimeProvider(API_KEY)
    device = provider.get_device(device_name)
    pm = generate_preset_pass_manager(optimization_level=1, target=device._backend.target)
    transpiled_circuit = pm.run(circ)
    job = device.run(transpiled_circuit, shots=shot_count, memory=True)
    return job._job_id


def get_job_results(job_id, length):
    API_KEY = os.getenv('IBM_APIKEY')
    provider = QiskitRuntimeService('ibm_quantum', API_KEY)
    job = provider.job(job_id)
    QBraidJob = QiskitJob(job_id, job)
    result = QBraidJob.result()

    rawData = result.measurements()

    unbrokenData = []
    for shot in rawData:
        for meas in shot:
            unbrokenData.append(meas)

    data = []
    for i in range(0, len(unbrokenData), length):
        data.append(''.join(map(str, unbrokenData[i:i+length])))
    
    return data


def write_data_to_file(data, file_path, labels):
    with open(file_path, 'a') as file:
        labelString = ''
        for label in labels:
            labelString += (label + ' ')
        labelString = labelString[:-1]
        for output in data:
            file.write(output + ' ' + labelString + '\n')
    file.close()

In [17]:
brisbane_id = launch_job('ibm_brisbane', 100000, circ)
brisbane_id

cv8h2ktemvv00084z7kg


In [22]:
kyiv_id = launch_job('ibm_kyiv', 100000, circ)
kyiv_id

'cv8h3x7kfn8g008vp7jg'

In [23]:
kyoto_id = launch_job('ibm_kyoto', 100000, circ)
kyoto_id

'cv8h410qrk1g008r0ffg'

In [24]:
sherbrooke_id = launch_job('ibm_sherbrooke', 100000, circ)
sherbrooke_id

'cv8h440qrk1g008r0fg0'

In [19]:
write_data_to_file(get_job_results(brisbane_id, 100), 'data/IBM_Superposition_QRNG_100qubit_large.txt', ['quantum', 'ibm_brisbane'])

In [25]:
write_data_to_file(get_job_results(kyoto_id, 100), 'data/IBM_Superposition_QRNG_100qubit_large.txt', ['quantum', 'ibm_kyoto'])

In [4]:
write_data_to_file(get_job_results('cv8h3x7kfn8g008vp7jg', 100), 'data/IBM_Superposition_QRNG_100qubit_large.txt', ['quantum', 'ibm_kyiv'])

In [27]:
write_data_to_file(get_job_results(sherbrooke_id, 100), 'data/IBM_Superposition_QRNG_100qubit_large.txt', ['quantum', 'ibm_sherbrooke'])

In [28]:
#Creating euql amount of non-quantum training data using numpy random module
with open('data/IBM_Superposition_QRNG_100qubit_large.txt', 'a') as file:
    file.write('\n')
    rng = np.random.default_rng()
    for i in range(400000):
        prng = rng.integers(0, 2, 100)
        rngString = ''.join(str(x) for x in prng)
        output = rngString + ' non-quantum N/A\n'
        file.write(output)
    
file.close()

Refactoring QRNG generation code for 10 qubit circuit